## Improved FNN by Cao

In [1]:
import os
import argparse
import logging
import time
import numpy as np
import numpy.random as npr
import matplotlib
matplotlib.use('agg')
import matplotlib.pyplot as plt

datafilepath = 'C:/Users/shiny/Documents/NeuralODE_MutantRodent/All_Rodent_concatenated_csv/RodentPaws_fps200.npy'

In [2]:
def load_data(datafilepath):
    data = np.load(datafilepath)
    traj_tot = data.copy()
    traj_tot = traj_tot[:,2000:9000,:]
    data = data[:, 2000:(7000+6*3*120), :]
    
    n = data.shape[1]
    start = 0
    stop = 0.25 + 1

    start_idx = 0
    stop_idx = n
    tot_idx = np.int64(data.shape[1]/120-start)

    orig_ts = np.linspace(start, stop-1, num=tot_idx)
    samp_ts = orig_ts[start_idx:stop_idx]

    orig_trajs = np.zeros((data.shape[0],data.shape[1],data.shape[2]))
    for i in range(data.shape[0]):
        for j in range(data.shape[2]):
            trajs = data[i,:,j]
            trajs_tot = traj_tot[i,:,j]
            orig_trajs[i,:,j] = (trajs - trajs_tot.mean()) / trajs_tot.std()
            samp_trajs = orig_trajs[:,start_idx:stop_idx,:]
    
    samp_trajs = samp_trajs.reshape(401, 7160, 8)


    return orig_trajs, samp_trajs, orig_ts, samp_ts


In [4]:
orig_trajs, samp_trajs, orig_ts, samp_ts = load_data(datafilepath)

In [5]:
samp_trajs = samp_trajs.reshape(401,7160,8)

In [6]:
X = samp_trajs[:50,0:500,:]

In [ ]:
import rpy2.robjects as ro
from rpy2.robjects.packages import importr
from rpy2.robjects import numpy2ri

nonlinearTseries = importr("nonlinearTseries")
data = numpy2ri.numpy2rpy(X)

cao_emb_dim = nonlinearTseries.estimateEmbeddingDim(
    data,  # time series
    len(data),  # number of points to use, use entire series
    6,  # time delay
    12,  # max no. of dimension
    0.95,  # threshold value
    0.1,  # max relative change
    True,  # do the plot
    "Computing the embedding dimension",  # main
    "dimension (d)",  # x_label
    "E1(d) & E2(d)",  # y_label
    ro.NULL,  # x_lim
    ro.NULL,  # y_lim
    0  # add a small amount of noise to the original series to avoid the
          # appearance of false neighbours due to discretization errors.
          # This also prevents the method to fail with periodic signals, 0 for no noise
)

embedding_dimension = int(cao_emb_dim[0])

In [13]:
embedding_dimension

-2147483648

In [28]:
plt.savefig('books_read.png')